# Weight generation script

This Jupyter Notebook contains functions for generating penalty weights for incorporation into an FBA-optimization. The output weight vectors, once collected into a .xls or .xlsx file with each column corresponding to a weight vector, can be used in the **FBA_Flux_Prediction.m** MATLAB script. To run this script, you will need: 

1. An SBML model in both .xml and .csv format and containing GPR terms formatted with each gene ID surrounded by parentheses, isozymes denoted by OR, members of a complex denoted by AND, and all isozymes and complexes surrounded by parentheses, as shown in the *fetchComponents()* function info below. Reactions in your model must end with a two character identifier indicating what tissue they map to (the script is currently setup to recognize **_lf** for leaf, **_st** for stem, and **_rt** for root. The functions will need to be modified to recognize others).
2. A .xls file containing expression tissue-specific expression data. See example file "ExpressionInput.xls" for an example of how this should be formatted.
3. A .xls file containing gene IDs for genes that show up in GPR terms in the model but that do not appear in your tissue-specific expression data file.

Versions of the weight generation function that include the median tissue-specific weight correction (getWeights) and that do not (getWeightsNoTissueSpecificMedianWeight) are both included below, along with a code cell that allows you to change the scaling factor list you want and, for each value, generate a new weight vector.

In [ ]:
# Set imports

import sys
sys.path

#!{sys.executable} -m pip install cobra
#!{sys.executable} -m pip install optlang

import cobra as cb
from cobra import Model, Reaction, Metabolite
import cobra.manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import gurobipy as gb
import optlang as opt
import math as math
import sympy as smp
cobra_config = cobra.Configuration()
cobra_config.solver = 'glpk'

sns.set()

In [ ]:
def fetchComponents(GPR):
    
    '''
    Takes a GPR string and breaks it up into individual parentheticals, which will represent either individual genes
    or terms, such as (G1ORG2). Returns an array of all the individual OUTERMOST parentheticals. So, (G1)OR(G2) will
    get returned as (G1),(G2), and ((G1)OR(G2))OR(G3) will come back as ((G1)OR(G2)),(G3)
    
    inputs:
    GPR: str representing a gene-reaction mapping
    
    outputs:
    parentheticals: an array of individual components
    relationship: the relationship between the returned components (either single, OR, or AND)
    
    '''
    thestring = str(GPR)
    parenCounter = 0
    parentheticals = np.array([])

    for i in range(len(thestring)):
        if thestring[i] == '(' and parenCounter == 0:
            startedParen = True
            parenCounter = parenCounter + 1
            startPos = i
        elif thestring[i] == '(':
            parenCounter = parenCounter + 1
        elif thestring[i] == ')':
            parenCounter = parenCounter - 1
            if parenCounter == 0 and startedParen == True:
                endPos = i+1
                parenthetical = thestring[startPos:endPos]
                parentheticals = np.append(parentheticals,parenthetical)

    for i in range(len(parentheticals)):
        thestring = thestring.replace(parentheticals[i],'',1)
    
    if thestring.find('AND') != -1:
        relationship = 'AND'
    elif thestring.find('OR') != -1:
        relationship = 'OR'
    else:
        relationship = 'SINGLE'
    
    return parentheticals,relationship,

In [ ]:
def getSingle(gene,geneExpression,tissue,unique):
    '''
    
    Takes a GPR, a transcript file, and a tissue. Returns the scale value if the GPR is a single gene, and otherwise
    calls the getAND or getOR functions to decompose the GPR.
    
    inputs: 
    gene: a str representing a GPR or gene-reaction mapping
    geneExpression: a pandas dataframe containing transcript information where each row is a gene and each column is a
    tissue denoted by a two character code (e.g. t2) that correspond to identical codes used as suffixes for reaction
    names
    tissue: the two character tissue code for the current reaction being worked on
    unique: DataFrame containing gene IDs for genes in model GPRs that are missing from expression dataset

    outputs: 
    scale: the scale/weight value calculated for the gene in question
    
    '''
    
    print('GetSingleModified')
    scale = -1
    inside = gene[1:-1]
    print('Inside:',inside)
    name = gene[1:-4]
    # check if it's on the missing list
    for z in range(len(unique)):
        if inside == unique[z]:
            scale = 'missing_on_unique'
            return scale
    for i in range(len(geneExpression.index)):
        if inside == geneExpression.iloc[i].name:
            values = np.array([])
            for q in range(len(geneExpression.iloc[i])):
                isBlank = geneExpression.iloc[i][q]
                print('Isblank',isBlank)
                if pd.isna(isBlank):
                    continue
                else:
                    values = np.append(values,geneExpression.iloc[i][q])
                    print('Values:',values)
            highestExpression = values.max()
            print('HighestExpression:',highestExpression)
            for n in range(len(geneExpression.columns)):
                print('Tissue:',tissue)
                print('Column',geneExpression.columns[n])
                if tissue == geneExpression.columns[n]:
                    #scaling
                    myValue = geneExpression.iloc[i][n]
                    if pd.isna(myValue):
                        scale = 'missing'
                    else:
                        print('My value:',geneExpression.iloc[i][n])
                        scale = highestExpression / geneExpression.iloc[i][n]
                    print('Scale:',scale)
                    break
    if scale == -1:
        components,relationship = fetchComponents(inside)
        if relationship == 'SINGLE':
            getSingle(components,geneExpression,tissue,unique)
        elif relationship == 'AND':
            scale = getAND(components,geneExpression,tissue,unique)
        elif relationship == 'OR':
            scale = getOR(components,geneExpression,tissue,unique)
    return scale

In [ ]:
def getAND(genes,geneExpression,tissue,unique):
    
    '''
    Takes the output from the fetchComponents() function and gets the individual values for each. Then takes the max,
    or worst, scale value and returns that
    
    inputs: 
    gene: a str representing a GPR or gene-reaction mapping
    geneExpression: a pandas dataframe containing transcript information where each row is a gene and each column is a
    tissue denoted by a two character code (e.g. t2) that correspond to identical codes used as suffixes for reaction
    names
    tissue: the two character tissue code for the current reaction being worked on
    unique: DataFrame containing gene IDs for genes in model GPRs that are missing from expression dataset

    
    outputs: 
    scale: the scale/weight value calculated for the set of genes in question
    '''
    
    
    scale = -1
    insides = np.array([])
    for i in range(len(genes)):
        todo = genes[i]
        insides = np.append(insides,todo)
    complexValues = np.array([])
    for p in range(len(insides)):
        value = getSingleModifiedForProtein(insides[p],geneExpression,tissue,unique)
        if value == 'missing_on_unique':
            scale = 'missing_on_unique'
        elif value != 'missing':
            complexValues = np.append(complexValues,value)
        try:
            scale = complexValues.max()
        except:
            scale = 'missing'
    return scale

In [ ]:
def getOR(genes,geneExpression,tissue,unique):
    
    '''
    Takes the output from the fetchComponents() function and gets the individual values for each. Then takes the mean
    of the scale values and returns that
    
    inputs: 
    gene: a str representing a GPR or gene-reaction mapping
    geneExpression: a pandas dataframe containing transcript information where each row is a gene and each column is a
    tissue denoted by a two character code (e.g. t2) that correspond to identical codes used as suffixes for reaction
    names
    tissue: the two character tissue code for the current reaction being worked on
    unique: DataFrame containing gene IDs for genes in model GPRs that are missing from expression dataset

    outputs: 
    scale: the scale/weight value calculated for the set of genes in question
    '''
    
    
    scale = -1
    insides = np.array([])
    for i in range(len(genes)):
        todo = genes[i]
        insides = np.append(insides,todo)
    complexValues = np.array([])
    for p in range(len(insides)):
        value = getSingleModifiedForProtein(insides[p],geneExpression,tissue,unique)
        if value == 'missing_on_unique':
            scale = 'missing_on_unique'
        elif value != 'missing':
            complexValues = np.append(complexValues,value)
        scale = complexValues.mean()
        if np.isnan(scale):
            scale = 'missing'
    return scale

In [ ]:
def getWeights(model_filename,model_csvname,geneExpression_filename,scaling_factor,uniqueFilename):
    
    '''
    This script generates a vector of weights that can then be exported to MATLAB to bias the minimization of absolute
    flux by using these weights as objective coefficients. It can work with GPRs of arbitrary complexity that consist
    of genes names connected with AND to denote membership in a protein complex and OR to denote isozyme pairs. 
    It assumes the 'worst' or heaviest penalty for a complex - members of an AND pair - is the correct value, but takes
    the mean for OR relationships. The getAND() and getOR() functions can be modified to change this logic if need be.
    The function also takes a scaling factor that attenuates the extent to which the expression evidence 
    affects the linear weight. Note that the function currently cannot handle expression values of 0. 
    
    Inputs:
    model_filename: str with SBML filename (e.g. 'dummy_model.xml')
    model_csvname: str with CSV model filename (e.g. 'dummy_model.csv')
    geneExpression_filename: str with geneExpression.xls filename (e.g. 'geneExpression.xls')
    scaling_factor: float from 0.0 to 1.0 
    uniqueFilename: str with .xls filename for file listing gene IDs in model GPRs missing from expression dataset
    
    Outputs: 
    scalingVector: a vector of length n, where n is the number of reactions in the model, 
    
    '''
    
    model = cb.io.read_sbml_model(model_filename)
    geneExpression = pd.read_excel(geneExpression_filename,index_col=0,header=0)
    raw = pd.read_csv(model_csvname,keep_default_na = False)
    unique = pd.read_excel(uniqueFilename,header=0)
    unique = unique['Gene']
    GPRarray = raw['GPR']
    
    scalingVector = np.array([])
    weightedVector = np.array([])
    
    for i in range(len(model.reactions)): # for each reaction in the model (i = each reaction)
        print('Iteration:',i)
        name = model.reactions[i].id
        print('Name:',name)
        tissue = name[-2:]
        print('Tissue:',tissue)
        GPR = GPRarray[i] # gene is set to the name of the gene corresponding to rxns i
        if GPR == '': # if that GPR is blank ...
            scalingVector = np.append(scalingVector,[1]) # append a 1 to the scaling vector and move on
            weightedVector = np.append(weightedVector, ['unweighted'])
        else:
            print('GPR:',GPR)
            initialScale = getSingle(GPR,geneExpression,tissue,unique)
            if initialScale == 'missing':
                print('missing')
                scalingVector = np.append(scalingVector,[1])
                weightedVector = np.append(weightedVector, ['weighted'])
            elif initialScale == 'missing_on_unique':
                print('missing')
                scalingVector = np.append(scalingVector, [1])
                weightedVector = np.append(weightedVector, ['unweighted'])
            else:
                finalScale = ((initialScale - 1) * scaling_factor)+1
                print('Final scale:',finalScale)
                scalingVector = np.append(scalingVector,finalScale)
                weightedVector = np.append(weightedVector, ['weighted'])
    allweights = scalingVector[weightedVector=='weighted']
    mean_weight = np.median(allweights)
    
    # now we need to get the median weight for leaf, stem, and root tissues independently. We can do that by iterating
    # through each reaction in all weights and for each one, adding it to either a leaf, stem, or root weight list by 
    # looking at the corresponding name in the model.reactions list.
    
    all_leaf_weights = []
    all_stem_weights = []
    all_root_weights = []
    
    for n in range(len(allweights)):
        reaction_name = model.reactions[n].id
        tissue = reaction_name[-2:]
        if tissue == 'lf':
            all_leaf_weights.append(allweights[n])
        elif tissue == 'st':
            all_stem_weights.append(allweights[n])
        elif tissue == 'rt':
            all_root_weights.append(allweights[n])
        else:
            continue
    
    leaf_median = np.median(all_leaf_weights)
    stem_median = np.median(all_stem_weights)
    root_median = np.median(all_root_weights)
    
    ## Now, instead of setting all unweighted ones equal to the overall median weight, we check whether to see if it's a leaf,
    ## stem or root reaction first and set to the appropriate median value
    for p in range(len(weightedVector)):
        if weightedVector[p] == 'unweighted':
            reaction_name = model.reactions[p].id
            tissue = reaction_name[-2:]
            if tissue == 'lf':
                scalingVector[p] = leaf_median
            elif tissue == 'st':
                scalingVector[p] = stem_median
            elif tissue == 'rt':
                scalingVector[p] = root_median
            else:
                scalingVector[p] = mean_weight

    return scalingVector, weightedVector

In [ ]:
def getWeightsNoTissueSpecificMedianWeight(model_filename,model_csvname,geneExpression_filename,scaling_factor,uniqueFilename):
    
    '''
    This script generates a vector of weights that can then be exported to MATLAB to bias the minimization of absolute
    flux by using these weights as objective coefficients. It can work with GPRs of arbitrary complexity that consist
    of genes names connected with AND to denote membership in a protein complex and OR to denote isozyme pairs. 
    It assumes the 'worst' or heaviest penalty for a complex - members of an AND pair - is the correct value, but takes
    the mean for OR relationships. The getAND() and getOR() functions can be modified to change this logic if need be.
    The function also takes a scaling factor from 0 to 1 that attenuates the extent to which the expression evidence 
    affects the linear weight. Note that the function currently cannot handle expression values of 0. 
    
    Inputs:
    model_filename: str with SBML filename (e.g. 'dummy_model.xml')
    model_csvname: str with CSV model filename (e.g. 'dummy_model.csv')
    geneExpression_filename: str with geneExpression xls filename (e.g. 'geneExpression.xls')
    scaling_factor: float from 0.0 to 1.0 
    uniqueFilename: str with .xls filename for file listing gene IDs in model GPRs missing from expression dataset

    
    Outputs: 
    scalingVector: a vector of length n, where n is the number of reactions in the model, 
    
    '''
    
    model = cb.io.read_sbml_model(model_filename)
    geneExpression = pd.read_excel(geneExpression_filename,index_col=0,header=0)
    raw = pd.read_csv(model_csvname,keep_default_na = False)
    unique = pd.read_excel(uniqueFilename,header=0)
    unique = unique['Gene']
    GPRarray = raw['GPR']
    
    scalingVector = np.array([])
    weightedVector = np.array([])
    
    for i in range(len(model.reactions)): # for each reaction in the model (i = each reaction)
        print('Iteration:',i)
        name = model.reactions[i].id
        print('Name:',name)
        tissue = name[-2:]
        print('Tissue:',tissue)
        GPR = GPRarray[i] # gene is set to the name of the gene corresponding to rxns i
        if GPR == '': # if that GPR is blank ...
            scalingVector = np.append(scalingVector,[1]) # append a 1 to the scaling vector and move on
            weightedVector = np.append(weightedVector, ['unweighted'])
        else:
            print('GPR:',GPR)
            initialScale = getSingle(GPR,geneExpression,tissue,unique)
            if initialScale == 'missing':
                print('missing')
                scalingVector = np.append(scalingVector,[1])
                weightedVector = np.append(weightedVector, ['weighted'])
            elif initialScale == 'missing_on_unique':
                print('missing')
                scalingVector = np.append(scalingVector, [1])
                weightedVector = np.append(weightedVector, ['unweighted'])
            else:
                finalScale = ((initialScale - 1) * scaling_factor)+1
                print('Final scale:',finalScale)
                scalingVector = np.append(scalingVector,finalScale)
                weightedVector = np.append(weightedVector, ['weighted'])
    allweights = scalingVector[weightedVector=='weighted']
    mean_weight = np.median(allweights)
    # now we need to get the median weight for leaf, stem, and root tissues independently. We can do that by iterating
    # through each reaction in all weights and for each one, adding it to either a leaf, stem, or root weight list by 
    # looking at the corresponding name in the model.reactions list.
    
    all_leaf_weights = []
    all_stem_weights = []
    all_root_weights = []
    
    for n in range(len(allweights)):
        reaction_name = model.reactions[n].id
        tissue = reaction_name[-2:]
        if tissue == 'lf':
            all_leaf_weights.append(allweights[n])
        elif tissue == 'st':
            all_stem_weights.append(allweights[n])
        elif tissue == 'rt':
            all_root_weights.append(allweights[n])
        else:
            continue
    
    leaf_median = np.median(all_leaf_weights)
    stem_median = np.median(all_stem_weights)
    root_median = np.median(all_root_weights)
    
    ## Now, instead of setting all unweighted ones equal to the overall median weight, we check whether to see if it's a leaf,
    ## stem or root reaction first and set to the appropriate median value
    for p in range(len(weightedVector)):
        if weightedVector[p] == 'unweighted':
            reaction_name = model.reactions[p].id
            tissue = reaction_name[-2:]
            if tissue == 'lf':
                scalingVector[p] = 1.0
            elif tissue == 'st':
                scalingVector[p] = 1.0
            elif tissue == 'rt':
                scalingVector[p] = 1.0
            else:
                scalingVector[p] = 1.0

    return scalingVector, weightedVector

## Automated script to generate a series of weights

In [ ]:
# making some weights for protein with normalization

# make a series of weights
weights = [0,1,1000]
#weights = [0,0,0.1,0.5,1,2,10,100,1000]
#weights = [0,1,10]
# loop through each of the weights and make each new file

modelname = 'example_model.xml'
geneExpressionName = 'Example_Omic_Data.xlsx'
modelcsvname = 'example_model.csv'
uniqueName = 'Example_Missing_Data.xlsx'

for i in range(len(weights)):
    name = 'M_Trans_Test_{0}.xls'.format(weights[i])
    scalingVector,weightedVector = getWeights(modelname,modelcsvname,geneExpressionName,weights[i],uniqueName)
    my_df = {'Scales':scalingVector,'Weighted':weightedVector}
    pd.DataFrame(my_df).to_excel(name)